In [ ]:
# Here we are using only the Dense Layer

# Control the output value using Relu Activation function and Predict the output by sigmoid function
# input_dim is the value 0f max lenth of padded sequence

# Step 1: Convert the Courpus data into Integer encoding
# Step 2: Build the Keras model using only Dense Layer

In [3]:
import pandas as pd

# Corpus 
positive = ['This is an excellent movie','The move was fantastic I like it',
            'You should watch it is brilliant','Exceptionally good','Wonderfully directed and executed I like it',
             'Its a fantastic series','Never watched such a brillent movie','It is a Wonderful movie']

negative = ["horrible acting",'waste of money','pathetic picture','It was very boring',
            'I did not like the movie','The movie was horrible','I will not recommend',
            'The acting is pathetic']

In [4]:
reviews = [(pr,"positive") for pr in positive] + [(nr,"negative") for nr in negative]

data = pd.DataFrame(reviews,columns=["comment",'review'])

# shuffle the DataFrame rows
data = data.sample(frac = 1)

data.head(3)

,comment,review
3,Exceptionally good,positive
9,waste of money,negative
1,The move was fantastic I like it,positive


In [5]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
data['review'] = lb_make.fit_transform(data["review"])

data.head(2)

,comment,review
3,Exceptionally good,1
9,waste of money,0


In [6]:
# lower the text
data['comment'] = data['comment'].str.lower()

# find max_length sequence

max_seq_len = max([len([y for y in x.split()]) for x in data['comment']])
max_seq_len

7

In [7]:
# number of samples
num_samples = data.shape[0]
num_samples

16

In [8]:
from tensorflow.keras.preprocessing import text

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(data['comment'])
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in data['comment']]

In [9]:
vocab_size = len(id2word)

vocab_size

44

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_inputs = pad_sequences(wids,padding='post')

padded_inputs

array([[14, 15,  0,  0,  0,  0,  0],
       [16, 17, 18,  0,  0,  0,  0],
       [ 3, 19,  6,  9,  4,  7,  1],
       [20,  5, 21, 22,  2,  0,  0],
       [23,  8,  9, 24,  0,  0,  0],
       [ 1,  6, 25, 26,  0,  0,  0],
       [10, 11,  0,  0,  0,  0,  0],
       [12, 27,  0,  0,  0,  0,  0],
       [ 3, 11,  5, 12,  0,  0,  0],
       [ 3,  2,  6, 10,  0,  0,  0],
       [ 4, 28, 13,  7,  3,  2,  0],
       [29, 30, 31,  8, 32,  2,  0],
       [ 4, 33, 13, 34,  0,  0,  0],
       [35, 36, 37, 38,  4,  7,  1],
       [ 1,  5,  8, 39,  2,  0,  0],
       [40, 41, 42,  1,  5, 43,  0]])

In [11]:
y = data['review'].values # output labels

y

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1])

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=7, activation='relu'))  # max_seq_len = 7 (Features)
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(padded_inputs, y, epochs=150, batch_size=10)
# evaluate the keras model

Epoch 1/150
2/2 [==============================] - 1s 0s/step - loss: 1.3147 - accuracy: 0.5000
Epoch 2/150
2/2 [==============================] - 0s 18ms/step - loss: 1.2428 - accuracy: 0.5000
Epoch 3/150
2/2 [==============================] - 0s 4ms/step - loss: 1.1692 - accuracy: 0.5000
Epoch 4/150
2/2 [==============================] - 0s 18ms/step - loss: 1.1072 - accuracy: 0.5000
Epoch 5/150
2/2 [==============================] - 0s 5ms/step - loss: 1.0614 - accuracy: 0.5000
Epoch 6/150
2/2 [==============================] - 0s 4ms/step - loss: 1.0180 - accuracy: 0.6250
Epoch 7/150
2/2 [==============================] - 0s 20ms/step - loss: 0.9699 - accuracy: 0.6250
Epoch 8/150
2/2 [==============================] - 0s 5ms/step - loss: 0.9362 - accuracy: 0.6250
Epoch 9/150
2/2 [==============================] - 0s 4ms/step - loss: 0.8909 - accuracy: 0.5625
Epoch 10/150
2/2 [==============================] - 0s 4ms/step - loss: 0.8639 - accuracy: 0.5625
Epoch 11/150
2/2 [=========

In [19]:
def predict_Sentiment(text):
    
    # lower the text
    text = text.lower()
    
    # Split the text
    token_text = text.split()
    
    # Convert to word to int
    word_2id = [word2id[x] for x in token_text]
    
    # pad the sequence
    
    padded_inputs = pad_sequences([word_2id],maxlen=7)
    print(padded_inputs.shape)
    print(padded_inputs)
    
    result = model.predict(padded_inputs)[0][0]
    
    if result>0.5:
        print("Review: Positive")
    else:
        print("Review: Negative")


In [20]:
text = 'good'

predict_Sentiment(text)

(1, 7)
[[ 0  0  0  0  0  0 15]]
Review: Positive


In [16]:
# Bad its new word 
# This is will not work for the words which are not part of the actual corpus data

text = 'bad'

predict_Sentiment(text)

KeyError: 'bad'